In [1]:
import os
import random
import numpy as np

import cv2
import albumentations as A
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, array_to_img, img_to_array

#import skimage.io as io

In [2]:
import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
%matplotlib inline

In [3]:
!pip install -q git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
from pycocotools.coco import COCO

In [4]:
"""
def getBinaryMask(imageObj, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)
        
        #Threshold because resizing may cause extraneous values
        new_mask[new_mask >= 0.5] = 1
        new_mask[new_mask < 0.5] = 0

        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask
"""

"\ndef getBinaryMask(imageObj, coco, catIds, input_image_size):\n    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)\n    anns = coco.loadAnns(annIds)\n    train_mask = np.zeros(input_image_size)\n    for a in range(len(anns)):\n        new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)\n        \n        #Threshold because resizing may cause extraneous values\n        new_mask[new_mask >= 0.5] = 1\n        new_mask[new_mask < 0.5] = 0\n\n        train_mask = np.maximum(new_mask, train_mask)\n\n    # Add extra dimension for parity with train_img size [X * X * 3]\n    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)\n    return train_mask\n"

In [5]:
"""
def dataGenerator(catIds,imgIds,coco,batchSize,path_images):
    while True:
        imgs_arr = np.zeros((batchSize, 256, 256, 3), dtype = np.float32)
        masks_arr = np.zeros((batchSize, 256, 256, 1), dtype = bool)
        for i in range(batchSize):
            image_dict = coco.loadImgs(random.choice(imgIds))[0]
            img = load_img(os.path.join(path_images,image_dict['file_name']), target_size = (256, 256, 3))
            msk = getBinaryMask(image_dict,coco,catIds,(256,256))

            imgs_arr[i] = img
            masks_arr[i] = msk
        yield (imgs_arr, masks_arr)
"""

"\ndef dataGenerator(catIds,imgIds,coco,batchSize,path_images):\n    while True:\n        imgs_arr = np.zeros((batchSize, 256, 256, 3), dtype = np.float32)\n        masks_arr = np.zeros((batchSize, 256, 256, 1), dtype = bool)\n        for i in range(batchSize):\n            image_dict = coco.loadImgs(random.choice(imgIds))[0]\n            img = load_img(os.path.join(path_images,image_dict['file_name']), target_size = (256, 256, 3))\n            msk = getBinaryMask(image_dict,coco,catIds,(256,256))\n\n            imgs_arr[i] = img\n            masks_arr[i] = msk\n        yield (imgs_arr, masks_arr)\n"

In [6]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,catIds,imgIds,coco,batchSize,path_images):
        self.catIds = catIds
        self.imgIds = imgIds
        self.coco = coco
        self.batchSize = batchSize
        self.path_images = path_images
    def __len__(self):
        return len(self.imgIds) // self.batchSize
    def getBinaryMask(self,imageObj, coco, catIds, input_image_size):
        annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        train_mask = np.zeros(input_image_size)
        for a in range(len(anns)):
            new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)

            #Threshold because resizing may cause extraneous values
            new_mask[new_mask >= 0.5] = 1
            new_mask[new_mask < 0.5] = 0

            train_mask = np.maximum(new_mask, train_mask)

        # Add extra dimension for parity with train_img size [X * X * 3]
        train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
        return train_mask
    def __getitem__(self,id_imgIds):
        imgs_arr = np.zeros((self.batchSize, 256, 256, 3), dtype = np.float32)
        masks_arr = np.zeros((self.batchSize, 256, 256, 1), dtype = bool)
        i = id_imgIds * self.batchSize
        id_imgIds_batch = self.imgIds[i:i + self.batchSize - 1]
        arr_index = 0
        for id_imgIds_batch_index in id_imgIds_batch:
            image_dict = self.coco.loadImgs(id_imgIds_batch_index)[0]
            img = load_img(os.path.join(self.path_images,image_dict['file_name']), target_size = (256, 256, 3)) 
            msk = self.getBinaryMask(image_dict,self.coco,self.catIds,(256,256))
            imgs_arr[arr_index] = img
            imgs_arr[arr_index] = imgs_arr[arr_index] / 255
            masks_arr[arr_index] = msk
            arr_index += 1
        return imgs_arr, masks_arr
    

In [7]:
def getImgsNamesList(dataDir='../input/coco-2017-dataset/coco2017/annotations',dataType=str(),classNames=str()):
    annFile = '{}/instances_{}.json'.format(dataDir,dataType)
    coco=COCO(annFile)
    catIds = coco.getCatIds(catNms=classNames)
    imgIds = coco.getImgIds(catIds=catIds)
    return catIds, imgIds, coco

In [8]:
catIds_train, imgIds_train, coco_train = getImgsNamesList(dataType='train2017',classNames='person')
catIds_val, imgIds_val, coco_val = getImgsNamesList(dataType='val2017',classNames='person')

loading annotations into memory...
Done (t=25.97s)
creating index...
index created!
loading annotations into memory...
Done (t=1.26s)
creating index...
index created!


In [9]:
len(imgIds_train)

64115

In [10]:
img_path_val = '../input/coco-2017-dataset/coco2017/val2017'
img_path_train = '../input/coco-2017-dataset/coco2017/train2017'

In [11]:
train_gen = DataGenerator(catIds_train,imgIds_train,coco_train,32,img_path_train)
val_gen = DataGenerator(catIds_val,imgIds_val,coco_val,32,img_path_val)

In [12]:
x, y = train_gen.__getitem__(1)
print(x.shape, y.shape)

(32, 256, 256, 3) (32, 256, 256, 1)


In [13]:
#for x, y in dataGenerator(catIds_train,imgIds_train,coco_train,16,img_path_train):
#    break

In [14]:
plt.imshow(array_to_img(x[4]))

In [15]:
plt.imshow(y[4])

In [16]:
#from tensorflow import keras
#from keras import layers
#from keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16

In [17]:
def jacard_coef(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

In [18]:
def conv_block(inputs,num_filters):
    x = Conv2D(num_filters,3,padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)  
    x = Conv2D(num_filters,3,padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)  
    return x
def define_decoder(inputs,skip_layer,num_filters):
    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    x = Conv2DTranspose(num_filters,(2,2),strides=(2,2),padding='same',kernel_initializer=init)(inputs)  
    g = tf.keras.layers.Concatenate()([x,skip_layer])
    g = conv_block(g,num_filters)
    return g
def vgg16_unet(input_shape):
    inputs = Input(shape=input_shape) 
    vgg16 = VGG16(include_top=False,weights='imagenet',input_tensor=inputs)  # We will extract encoder layers based on their output shape from vgg16 model  s1 = vgg16.get_layer('block1_conv2').output  
    s1 = vgg16.get_layer('block1_conv2').output
    s2 = vgg16.get_layer('block2_conv2').output  
    s3 = vgg16.get_layer('block3_conv3').output  
    s4 = vgg16.get_layer('block4_conv3').output    # bottleneck/bridege layer from vgg16
    b1 = vgg16.get_layer('block5_conv3').output #32
  
  # Decoder Block
    d1 = define_decoder(b1,s4,512)
    d2 = define_decoder(d1,s3,256)
    d3 = define_decoder(d2,s2,128)
    d4 = define_decoder(d3,s1,64)  #output layer
    outputs = Conv2D(1,1,padding='same',activation='sigmoid')(d4)
    model = Model(inputs,outputs)
  
    return model

In [19]:
model = vgg16_unet(input_shape = (256,256,3))
model.summary()

2022-06-18 09:48:39.917207: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


   16384/58889256 [..............................] - ETA: 0s

2022-06-18 09:48:40.028752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 09:48:40.030128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 09:48:40.032051: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-18 09:48:40.032502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
_____________________________

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=[jacard_coef_loss], metrics=['accuracy', jacard_coef])
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint('./model_epoch_{epoch:00d}', save_best_only= False)
callbacks = [checkpoint,reduce_lr]

In [21]:
epochs = 15
model_history = model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

2022-06-18 09:48:44.247195: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-06-18 09:48:47.619231: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2003/2003 [==============================] - 1688s 834ms/step - loss: -0.4954 - accuracy: 0.8787 - jacard_coef: 0.4954 - val_loss: -0.5703 - val_accuracy: 0.9156 - val_jacard_coef: 0.5703


2022-06-18 10:16:54.327037: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/15
2003/2003 [==============================] - 1533s 765ms/step - loss: -0.5921 - accuracy: 0.9158 - jacard_coef: 0.5921 - val_loss: -0.5893 - val_accuracy: 0.9072 - val_jacard_coef: 0.5893
Epoch 3/15
2003/2003 [==============================] - 1538s 767ms/step - loss: -0.6274 - accuracy: 0.9257 - jacard_coef: 0.6274 - val_loss: -0.6340 - val_accuracy: 0.9250 - val_jacard_coef: 0.6340
Epoch 4/15
2003/2003 [==============================] - 1593s 795ms/step - loss: -0.6513 - accuracy: 0.9319 - jacard_coef: 0.6513 - val_loss: -0.6284 - val_accuracy: 0.9329 - val_jacard_coef: 0.6284
Epoch 5/15
2003/2003 [==============================] - 1558s 778ms/step - loss: -0.6676 - accuracy: 0.9359 - jacard_coef: 0.6676 - val_loss: -0.6614 - val_accuracy: 0.9366 - val_jacard_coef: 0.6614
Epoch 6/15
2003/2003 [==============================] - 1572s 784ms/step - loss: -0.6802 - accuracy: 0.9389 - jacard_coef: 0.6802 - val_loss: -0.6675 - val_accuracy: 0.9362 - val_jacard_coef: 0.6675
Epoch

In [22]:
"""
import tensorflow as tf
from keras.layers import *


model = Sequential()

model.add(Lambda(lambda x: x / 255))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(256, 256, 3)))
model.add(Dropout(0.1))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())

#model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
#model.add(Dropout(0.1))
#model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
#model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())
 
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
model.add(Dropout(0.1))
model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))
model.add(Dropout(0.1))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())


model.add(Conv2D(1, (1,1), activation = 'sigmoid'))

#model.compile(loss = 'binary_crossentropy', metrics = 'accuracy')
#model.compile(loss = 'binary_crossentropy', metrics = [tf.keras.metrics.IoU(num_classes=1)])
model.compile(optimizer='adam',loss = [jacard_coef_loss], metrics = [jacard_coef,'accuracy'])
"""

"\nimport tensorflow as tf\nfrom keras.layers import *\n\n\nmodel = Sequential()\n\nmodel.add(Lambda(lambda x: x / 255))\nmodel.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(256, 256, 3)))\nmodel.add(Dropout(0.1))\nmodel.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))\nmodel.add(BatchNormalization())\n\n#model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))\n#model.add(Dropout(0.1))\n#model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))\n#model.add(BatchNormalization())\n\nmodel.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', ))\nmodel.add(Dropout(0.1))\nmodel.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))\nmodel.add(BatchNormalization())\n \nmodel.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_no

In [23]:
#import keras
#model = keras.models.load_model('../input/best-model-coco/model_jacard_05446',custom_objects={'jacard_coef':jacard_coef,'jacard_coef_loss':jacard_coef_loss})

In [24]:
#checkpoint = ModelCheckpoint("./", monitor='val_jacard_coef', verbose=1, save_best_only=True, mode='max')

In [25]:
"""
batch_size = 16
model.fit_generator(dataGenerator(catIds_train,imgIds_train,coco_train,batch_size,img_path_train),
              steps_per_epoch=25,
              epochs=5000,
              verbose=1,
              callbacks=[checkpoint],
              validation_data=dataGenerator(catIds_val,imgIds_val,coco_val,batch_size,img_path_val),
              validation_steps=32,
              class_weight=None,
              max_queue_size=10,
              workers=1,
              use_multiprocessing=False,
              shuffle=True
              #initial_epoch=0
              )
"""

'\nbatch_size = 16\nmodel.fit_generator(dataGenerator(catIds_train,imgIds_train,coco_train,batch_size,img_path_train),\n              steps_per_epoch=25,\n              epochs=5000,\n              verbose=1,\n              callbacks=[checkpoint],\n              validation_data=dataGenerator(catIds_val,imgIds_val,coco_val,batch_size,img_path_val),\n              validation_steps=32,\n              class_weight=None,\n              max_queue_size=10,\n              workers=1,\n              use_multiprocessing=False,\n              shuffle=True\n              #initial_epoch=0\n              )\n'

In [26]:
def dataGenerator_test(batchSize,path_images):
    while True:
        imgs_arr = np.zeros((batchSize, 256, 256, 3), dtype = np.float32)
        image_list = os.listdir(path_images)
        for i in range(batchSize):
            image_name = random.choice(image_list)
            img = load_img(os.path.join(path_images,image_name), target_size = (256, 256, 3))
            imgs_arr[i] = img
        yield imgs_arr

In [27]:
def single_image(path):
    imgs_arr = np.zeros((1, 256, 256, 3), dtype = np.float32)
    img = load_img(path, target_size = (256, 256, 3))
    imgs_arr[0] = img
    return imgs_arr

In [28]:
#import keras
#model = tf.keras.models.load_model('../input/model_epoch_14',custom_objects={'jacard_coef':jacard_coef,'jacard_coef_loss':jacard_coef_loss})

In [29]:
#path_images = '../input/coco-2017-dataset/coco2017/test2017'
#for x in dataGenerator_test(16,path_images):
#    break

In [30]:
x = single_image('../input/kostya/kostya.jpeg')

In [31]:
img_index = 0

In [32]:
pred = model.predict(x[img_index].reshape((1, 256, 256,3)))

In [33]:
plt.imshow(array_to_img(x[img_index]))

In [34]:
plt.imshow(array_to_img(pred.reshape((256, 256, 1))))